<img style="float: left; margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="300" height="500" /> 
    
    
### <font color='navy'> Simulación de procesos financieros. 

**Nombres:** Daniela Michel Mercado y Mateo Verea Dorantes


**Fecha:** 22 de noviembre del 2021

**Expediente** : 724506.
**Profesor:** Oscar David Jaramillo Zuluaga.
    
**Link Github**: https://github.com/danielamichel/Tarea7-8_DMichel_MVerea/blob/main/Tarea%208.ipynb 
# Tarea 8: Clase 22


Replicar el procedimiento anterior para valoración de opciones 'call', pero en este caso para opciones tipo 'put' y además deben de comparar con los resultados teoricos del método de valuación de Black-Scholes (ecuación del principio de clase)

### Ejercicio 1:

Código de solución estudiante 1

In [ ]:
# Código de solución estudiante 1
.
.
.
.


Código de solución estudiante 2

In [ ]:
# Código de solución estudiante 2
.
.
.
.
